# SQL Alchemy

> Alchemy related functions.

In [ ]:
#| default_exp alchemy

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti
import os

from typing import List
from tqdm import tqdm

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from deepface.commons import functions

from sql_face.tables import Base, Image, Detector, CroppedImage, EmbeddingModel, FaceImage, QualityModel, QualityImage

## Connect session
Connect to SQL database

In [ ]:
#| export

def get_session(
    output_dir:str, # Output directory
    db_name:str, # .db file name
                    ):  
    db_path = os.path.join(output_dir,db_name+'.db')       
    engine = create_engine(f"sqlite:///{db_path}")
    if not os.path.exists(db_path):
        if not os.path.exists(output_dir):
            print(f'Creating output directory at {output_dir}')
            os.mkdir(output_dir)
            
        print(f'Creating Db file at {db_path}')
        Base.metadata.create_all(engine)

    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    return session

## Table creators 
Detectors, Images, Cropped Images, etc.

In [ ]:
#| export

def create_detectors(
                session,
                detector_names: List[str]
                ):
        
        for detector_entry in detector_names:
                detector = (
                        session.query(Detector)
                        .filter(Detector.name == detector_entry)
                        .one_or_none()
                        )
                if detector is None:
                        detector = Detector(name=detector_entry)
                        session.add(detector)
                session.commit()

In [ ]:
#| export

def create_embedding_models(session,
                            embedding_model_names: List[str]):
        
        for emb_model_entry in embedding_model_names:
            emb_model = (
                session.query(EmbeddingModel)
                    .filter(EmbeddingModel.name == emb_model_entry)
                    .one_or_none()
            )
            if emb_model is None:
                emb_model = EmbeddingModel(name=emb_model_entry)
                session.add(emb_model)
        session.commit()


In [ ]:
#| export

def create_quality_models(session,
                            quality_model_names: List[str]):
        
        for qua_model_entry in quality_model_names:
            qua_model = (
                session.query(QualityModel)
                    .filter(QualityModel.name == qua_model_entry)
                    .one_or_none()
            )
            if qua_model is None:
                qua_model = QualityModel(name=qua_model_entry)
                session.add(qua_model)
        session.commit()

In [ ]:
#| export

def fill_cropped_image_general(cr_img: CroppedImage, **kwargs):
    image = cr_img.images.get_image()      
    
        
    # todo : maybe save the image cropped somewhere?
    try:
        img_cropped, bounding_box = functions.preprocess_face(img=image,
                                                                detector_backend=cr_img.detectors.name,
                                                                enforce_detection=True,
                                                                return_region=True)
        
        cr_img.bounding_box = bounding_box
        cr_img.face_detected = True

    except ValueError:
        cr_img.bounding_box = []
        cr_img.face_detected = False
        # todo: change warning if the image is a video(frame).
        print(f'Face not found in {cr_img.images.path} with {cr_img.detectors.name}')



In [ ]:
#| export

def create_cropped_images(session, 
                ):
        
        all_detectors = (session.query(Detector).all())
        for det in all_detectors:

            # Load SERFIQ model if neccesary
            # TODO: serfiq detector not implemented.
            if det.name == 'mtcnn_serfiq':
                ser_fiq = self.ser_fiq
                fill_cropped_image = self.fill_cropped_image_serfiq
            else:
                ser_fiq = None
                fill_cropped_image = fill_cropped_image_general

            subquery = session.query(CroppedImage.image_id) \
                .filter(CroppedImage.detectors == det)
            images = (
                session.query(Image)
                    .filter(Image.image_id.notin_(subquery))
                    .all()
            )

            for img in tqdm(images[:5], desc=f'TRIM Creating Cropped Images for detector {det.name}'):
                cropped_image = CroppedImage()
                cropped_image.images = img
                cropped_image.detectors = det
                fill_cropped_image(cropped_image, ser_fiq = ser_fiq)
                session.add(cropped_image)
                session.commit()

In [ ]:
#| export
def create_face_images(session):
        
    all_embedding_models = (session.query(EmbeddingModel).all())
    for emb in tqdm(all_embedding_models, desc='Embedding models'):
        subquery = session.query(FaceImage.croppedImage_id) \
            .filter(FaceImage.embeddingModels == emb)
        cropped_images = (
            session.query(CroppedImage) \
                .filter(CroppedImage.croppedImage_id.notin_(subquery),
                        CroppedImage.face_detected == True)
                .all()
        )

        for cr_img in tqdm(cropped_images, desc=f'Face images in {emb.name}'):
            face_image = FaceImage()
            face_image.croppedImages = cr_img
            face_image.embeddingModels = emb
            session.add(face_image)
            session.commit()

In [ ]:
#| export

def create_quality_images(session):
        
        all_quality_models = (session.query(QualityModel).all())
        for qua in tqdm(all_quality_models, desc='Quality models'):
            subquery = session.query(QualityImage.faceImage_id) \
                .filter(QualityImage.qualityModels == qua)
            face_images = (
                session.query(FaceImage) \
                    .filter(FaceImage.faceImage_id.notin_(subquery))
                    .all()
            )

            for face_img in tqdm(face_images, desc=f'Quality images in {qua.name}'):
                qua_image = QualityImage()
                qua_image.faceImages = face_img
                qua_image.qualityModels = qua
                session.add(qua_image)
                session.commit()

# Table attribute update
Once the tables are created, functions to fill in the Image, CroppedImage, FaceImage and Qualityimage attributes.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()